# Testing debilis percent variance explained gen 1 to gen 6

## Setup

In [1]:
import os
import sys
import glob
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import re
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [3]:
from cvtk.cvtk import TemporalFreqs, TiledTemporalFreqs
from cvtk.cov import stack_temporal_covariances
import cvtk.variant_files as vf
from cvtk.gintervals import GenomicIntervals
from cvtk.pca import FreqPCA
from cvtk.bootstrap import bootstrap_ci
from cvtk.plots import rep_plot_pca, correction_diagnostic_plot
from cvtk.utils import integerize
from cvtk.utils import extract_empirical_nulls_diagonals, extract_temporal_cov_diagonals
from cvtk.cov import stack_replicate_covariances, stack_temporal_covs_by_group
from cvtk.variant_files import VCFFile

In [4]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.figsize'] = (8.0, 4.0)
mpl.rcParams['figure.dpi'] = 200

## Varianta Data Loading

### Load in TSV data and reshape

In [5]:
keep_seqids = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17']
keep_autos = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17']

In [520]:
rawdata = pd.read_csv('../../ann3_1_6_snps_cvtk.v1.tsv', delimiter='\t')
samples = ['1_BFL', '6_BFL', '1_KPC', '6_KPC', '1_LBJ', '6_LBJ', '1_HCC', '6_HCC']
freqs = rawdata[[f"pr_{samp}" for samp in samples]].values.T
depths = rawdata[[f"reads_{samp}" for samp in samples]].values.T

In [521]:
gi = GenomicIntervals()
for row in rawdata.itertuples(index=False):
    seqid = row[0].replace('Ha412HOChr', '')
    gi.append(seqid, int(row[1]))

In [522]:
gi.infer_seqlens()

In [523]:
samples = [('BFL', 1), ('BFL', 6), ('KPC', 1), ('KPC', 6), ('LBJ', 1), ('LBJ', 6), ('HCC', 1), ('HCC', 6)]


## Replicate Covariance Analysis

In [524]:
tiles = GenomicIntervals.from_tiles(gi.seqlens, width=5e7)

In [525]:
d = TiledTemporalFreqs(tiles, freqs=freqs, depths=depths, diploids=500, gintervals=gi, samples=samples)

In [526]:
d.samples

[('BFL', 1),
 ('BFL', 6),
 ('HCC', 1),
 ('HCC', 6),
 ('KPC', 1),
 ('KPC', 6),
 ('LBJ', 1),
 ('LBJ', 6)]

In [527]:
d.freqs.shape

(4, 2, 3086)

In [528]:
covs_cis = d.bootstrap_cov(B=5000, progress_bar=True, average_replicates=False)

In [529]:
covs_cis

array([[[0.05582853, 0.04672303, 0.0321395 , 0.06676118],
        [0.04672303, 0.10736193, 0.03735812, 0.06944588],
        [0.0321395 , 0.03735812, 0.0455248 , 0.03807384],
        [0.06676118, 0.06944588, 0.03807384, 0.09545273]],

       [[0.06594268, 0.06099336, 0.04541572, 0.07367628],
        [0.06099336, 0.12629512, 0.04975013, 0.07929739],
        [0.04541572, 0.04975013, 0.06127427, 0.0466415 ],
        [0.07367628, 0.07929739, 0.0466415 , 0.10437607]],

       [[0.07690963, 0.07848474, 0.05361482, 0.08748124],
        [0.07848474, 0.14334201, 0.06156005, 0.09629043],
        [0.05361482, 0.06156005, 0.07662499, 0.05900329],
        [0.08748124, 0.09629043, 0.05900329, 0.1201737 ]]])

## Genome-wide Covariances

In [530]:
gw_covs = d.calc_cov(bias_correction=True)

In [531]:
gw_covs

array([[0.06594268, 0.06099336, 0.04541572, 0.07367628],
       [0.06099336, 0.12629512, 0.04975013, 0.07929739],
       [0.04541572, 0.04975013, 0.06127427, 0.0466415 ],
       [0.07367628, 0.07929739, 0.0466415 , 0.10437607]])

### Convergence G

A simple estimate:

In [532]:
# this is specific to this design
def rep_G(covs, standardize=True):
    out = ((covs[0,1]+ covs[1,2] + covs[0,2] + covs[0,3] + covs[1,3] + covs[2,3])/6)
    if standardize:
        return out/(np.diag(covs).mean())
    return out

rep_G(gw_covs)

0.6627292028938732

In [533]:
windowed_covs = d.calc_cov_by_tile()                            

In [534]:
B = 5000

windowed_rep_G_num = np.array([rep_G(win, False) for win in windowed_covs])
windowed_rep_G_denom = np.array([np.diag(win).mean() for win in windowed_covs])

straps = []
for b in np.arange(B):
    bidx = np.random.randint(0, len(windowed_covs), len(windowed_covs))
    g = windowed_rep_G_num[bidx].mean() / windowed_rep_G_denom[bidx].mean()
    straps.append(g)
    
print(bootstrap_ci(rep_G(gw_covs), straps))

[0.58322454 0.6627292  0.72787547]


In [420]:
tiles

GenomicIntervals — 17 unique seqids, 306 features
             GenomicInterval
0         01:[0, 10000000.0)
1  01:[10000000, 20000000.0)
2  01:[20000000, 30000000.0)
3  01:[30000000, 40000000.0)
4  01:[40000000, 50000000.0)
[ 301 more GenomicIntervals ]